In [516]:
# Create a simple Point class to represent the affine points.
from collections import namedtuple
Point = namedtuple("Point", "x y")
Point_3d = namedtuple("Point_3d", "x y z")

In [517]:
# Point addition in affine coordinate :

def add_affine(P,Q,a,p) :

    if P==None :
        return Q
    elif Q==None :
        return P
    elif P==Q : 
        l = (3 * P.x**2 + a) * pow((2 * P.y),-1,p)

    else :
        if P.x != Q.x : # Check if P+Q != Infinity (which means P and Q are symetric)
            l = (Q.y - P.y) * pow((Q.x - P.x),-1,p)
        else :
            return None
    x = (l**2 - P.x - Q.x) % p
    y = (l * (P.x - x) - P.y) % p

    return Point(x,y)

In [518]:
# Point doubling in affine coordinate :

def double_affine(P,a,p) :

    if P==None :
        return None
    else :
        if P.y != 0 : # Check if P.y = 0 (which means 2*P = Infinity)
            l = (3 * P.x**2 + a) * pow((2 * P.y),-1,p)
        else :
            return None
    x = (l**2 - P.x - P.x) % p
    y = (l * (P.x - x) - P.y) % p

    return Point(x,y)

In [519]:
# Point multiplication in affine coordinate :

def double_and_add(n,P,a,p):
    result = P
    for b in format(n,'b')[1:]:
        result = double_affine(result,a,p) # Doubling
        if b=='1':
            result = add_affine(result,P,a,p) # Addition
    return result

In [637]:
# Affine to Jacobi conversion :

def affine_to_jacobi(P,p) :
    return Point_3d(P.x,P.y,1)

# Affine to Jacobi_MD conversion :

def affine_to_jacobi_MD(P,p,prec) :
    r = (2**prec) % p
    r2 = (r*r)%p
    rinv = rinv = pow(r,-1,p) #rinv = inverse_mod(r,p)
    
    X_MD = (P.x*r)%p #MontMult(P.x,r2,p)   # a -> a_M = a * R
    Y_MD = (P.y*r2*rinv)%p #MontMult(P.y,r2,p)
    Z_MD = r%p; #MontR(p)
    return Point_3d(X_MD,Y_MD,Z_MD)

# Jacobi to Affine conversion :

def jacobi_to_affine(P,p) :
    
    if P == None or P.z == 0 :
        return None
    
    zinv = pow(P.z,-1,p)
    zinv2 = (zinv*zinv) %p
    zinv3 = (zinv2*zinv) %p
    
    x_affine = (P.x*zinv2) %p
    y_affine = (P.y*zinv3) %p
    return Point(x_affine,y_affine)

# Jacobi_MD to Affine conversion :

def jacobi_MD_to_affine(P_MD,p,prec) :
    
    if P_MD == None or P_MD.z == 0 :
        return None
    
    r = (2**prec) % p
    r2 = (r*r)%p
    rinv = pow(r,-1,p) #rinv = inverse_mod(r,p)
    
    zinv_MD = (pow(P_MD.z,-1,p) * r2) % p # zinv_MD = inverse_mod(P.z,p) # P.z is on Montgomery domain
    zinv2_MD = (zinv_MD*zinv_MD*rinv) %p
    zinv3_MD = (zinv2_MD*zinv_MD*rinv) %p
    
    x_affine_MD = (P_MD.x*zinv2_MD*rinv) %p # P.x is on Montgomery domain
    y_affine_MD = (P_MD.y*zinv3_MD*rinv) %p # P.y is on Montgomery domain
    
    x_affine = (x_affine_MD * rinv) %p
    y_affine = (y_affine_MD * rinv) %p
    
    return Point(x_affine,y_affine)

In [525]:
# Point doubling in Jacobian coordinate (Simplified formula)

def double_jacobi(P,a,p) :
    
    if P == None or P.z==0 : # z = 0 means point at Infinity
        return None
    
    X1=P.x
    Y1=P.y
    Z1=P.z
    XX = (X1*X1) % p
    YY = (Y1**2) % p
    YYYY =(YY**2) % p
    ZZ = (Z1**2) % p
    S1 = (X1+YY) % p
    S2 = (S1**2) % p
    S3 = (S2-XX) % p
    S4 = (S3-YYYY) % p
    S = (2*S4) % p
    ZZ2 =( ZZ**2) % p
    M1 = (3*XX) % p
    M2 = (a*ZZ2) % p
    M = (M1+M2) % p
    T1 = (M**2) % p
    T2 = (2*S) % p
    X3 = (T1-T2) % p
    Y31 =( 8*YYYY) % p
    Y32 =( S-X3) % p
    Y33 =( M*Y32) % p
    Y3 = (Y33-Y31) % p
    Z31 =( Y1+Z1) % p
    Z32 =( Z31**2) % p
    Z33 =( Z32-YY) % p
    Z3 = (Z33-ZZ) % p
    return Point_3d(X3,Y3,Z3)

# Point doubling in Jacobian coordinate

def double_jacobi_old(P,a) :
    
    if P == None :
        return None
    
    X1=P.x
    Y1=P.y
    Z1=P.z
    XX = X1**2
    YY = Y1**2
    YYYY = YY**2
    ZZ = Z1**2
    S = 2*((X1+YY)**2-XX-YYYY)
    M = 3*XX+a*ZZ**2
    T = M**2-2*S
    X3 = T
    Y3 = M*(S-T)-8*YYYY
    Z3 = (Y1+Z1)**2-YY-ZZ
    return Point_3d(X3,Y3,Z3)

In [563]:
def double_jacobi_MD(P_MD,a,p,prec) :
    
    if P == None or P_MD.z==0 : # z = 0 means point at Infinity
        return None
    R = (2**prec) % p
    r2 = (R*R)%p
    rinv = pow(R,-1,p) #rinv = inverse_mod(r,p)
    
    X1=P_MD.x
    Y1=P_MD.y
    Z1=P_MD.z
    XX = (X1*X1*rinv) % p # Mult(X1,X1)
    YY = (Y1*Y1*rinv) % p
    YYYY =(YY*YY*rinv) % p
    ZZ = (Z1*Z1*rinv) % p
    S1 = (X1+YY) % p
    S2 = (S1*S1*rinv) % p
    S3 = (S2-XX) % p
    S4 = (S3-YYYY) % p
    S = (2*R*S4*rinv) % p
    ZZ2 =(ZZ*ZZ*rinv) % p
    M1 = (3*R*XX*rinv) % p
    M2 = (a*R*ZZ2*rinv) % p
    M = (M1+M2) % p
    T1 = (M*M*rinv) % p
    T2 = (2*R*S*rinv) % p
    X3 = (T1-T2) % p
    Y31 =(8*R*YYYY*rinv) % p
    Y32 =(S-X3) % p
    Y33 =(M*Y32*rinv) % p
    Y3 = (Y33-Y31) % p
    Z31 =(Y1+Z1) % p
    Z32 =(Z31*Z31*rinv) % p
    Z33 =(Z32-YY) % p
    Z3 = (Z33-ZZ) % p
    return Point_3d(X3,Y3,Z3)

In [582]:
# Point addition in Jacobian coordinate
    
def add_jacobi(P,Q,a,p) :
    
    # r = 2**prec; r2 = (r*r) % p; rinv = inverse_mod(r,p);
    
    if (P==None or P.z==0) and (Q==None or Q.z==0) :
        return None
    if P==None or P.z==0 : # z=0 means that the point is at infinity
        return Q
    if Q==None or Q.z==0 :
        return P
    if P==Q :
        return double_jacobi(P,a,p)
    X1 = P.x
    Y1 = P.y
    Z1 = P.z
    
    X2 = Q.x
    Y2 = Q.y
    Z2 = Q.z
        
    Z1Z1 = (Z1*Z1) % p
    #  Z1Z1 = (Z1*Z1*rinv) % p
    Z2Z2 = (Z2**2) % p
    U1 = (X1*Z2Z2) % p
    U2 = (X2*Z1Z1) % p
    S11 = (Y1*Z2) % p
    S1 = (S11*Z2Z2) % p
    S21 = (Y2*Z1) % p
    S2 = (S21*Z1Z1) % p
    H = (U2-U1) % p
    # H = U2-U1                            #in MWMAC: ModSub(U2,U1,p)
    I1 = (2*H) % p
    # two_MD = (2*r2*rinv) % p             #in MWMAC: MontMult(2,r2,p)
    # I1 = (two_MD * H * rinv) % p
    I = (I1**2) % p
    J = (H*I) % p
    r1 = (S2-S1) % p
    r = (2*r1) % p
    V = (U1*I) % p
    X31 = (r**2) % p
    X32 = (2*V) % p
    X33 = (X31-J) % p
    X3 = (X33-X32) % p
    Y31 = (V-X3) % p
    Y32 = (2*S1) % p
    Y33 = (r*Y31) % p
    Y34 = (Y32*J) % p
    Y3 = (Y33-Y34) % p
    Z31 = (Z1+Z2) % p
    Z32 = (Z31**2) % p
    Z33 = (Z32-Z1Z1) % p
    Z34 = (Z33-Z2Z2) % p
    Z3 = (Z34*H) % p
    return Point_3d(X3,Y3,Z3)

In [583]:
# Point addition in Jacobian coordinate using Montgomery arithmetic
   
def add_jacobi_MD(P_MD,Q_MD,a,p,prec) :
    
    R = (2**prec) % p
    r2 = (R*R)%p
    rinv = pow(R,-1,p) #rinv = inverse_mod(r,p)
    
    if (P_MD==None or P_MD.z==0) and (Q_MD==None or Q_MD.z==0) :
        return None
    if P_MD==None or P_MD.z==0 : # z=0 means that the point is at infinity
        return Q_MD
    if Q_MD==None or Q_MD.z==0 :
        return P_MD
    if P_MD==Q_MD :
        return double_jacobi_MD(P_MD,a,p,prec)
    X1 = P_MD.x
    Y1 = P_MD.y
    Z1 = P_MD.z
    
    X2 = Q_MD.x
    Y2 = Q_MD.y
    Z2 = Q_MD.z
        
    Z1Z1 = (Z1*Z1*rinv) % p
    Z2Z2 = (Z2*Z2*rinv) % p
    U1 = (X1*Z2Z2*rinv) % p
    U2 = (X2*Z1Z1*rinv) % p
    S11 = (Y1*Z2*rinv) % p
    S1 = (S11*Z2Z2*rinv) % p
    S21 = (Y2*Z1*rinv) % p
    S2 = (S21*Z1Z1*rinv) % p
    H = (U2-U1) % p
    I1 = (2*R*H*rinv) % p
    I = (I1*I1*rinv) % p
    J = (H*I*rinv) % p
    r1 = (S2-S1) % p
    r = (2*R*r1*rinv) % p
    V = (U1*I*rinv) % p
    X31 = (r*r*rinv) % p
    X32 = (2*R*V*rinv) % p
    X33 = (X31-J) % p
    X3 = (X33-X32) % p
    Y31 = (V-X3) % p
    Y32 = (2*R*S1*rinv) % p
    Y33 = (r*Y31*rinv) % p
    Y34 = (Y32*J*rinv) % p
    Y3 = (Y33-Y34) % p
    Z31 = (Z1+Z2) % p
    Z32 = (Z31*Z31*rinv) % p
    Z33 = (Z32-Z1Z1) % p
    Z34 = (Z33-Z2Z2) % p
    Z3 = (Z34*H*rinv) % p
    return Point_3d(X3,Y3,Z3)

In [588]:
a = 11
p=43
prec = 512
P = Point(14,2)
Q = Point(3,23)

P_jac = affine_to_jacobi(P,p)
Q_jac = affine_to_jacobi(Q,p)

P_MD = affine_to_jacobi_MD(P,p,prec)
Q_MD = affine_to_jacobi_MD(Q,p,prec)

R0 = add_affine(P,Q,a,p)
R1_jac = add_jacobi(P_jac,Q_jac,a,p)
R1 = jacobi_to_affine(R1_jac,p)
R2_MD = add_jacobi_MD(P_MD,Q_MD,a,p,prec)
R2 = jacobi_MD_to_affine(R2_MD,p,prec)

print("Result using Affine : ", R0)
print("Result using Jacobi : ", R1)
print("Result using Montgomery : ", R2)

Result using Affine :  Point(x=30, y=9)
Result using Jacobi :  Point(x=30, y=9)
Result using Montgomery :  Point(x=30, y=9)


In [569]:
def add_jacobi_old(P,Q,a) :
    
    if P==None :
        return Q
    if Q==None :
        return P
    
    X1 = P.x
    Y1 = P.y
    Z1 = P.z
    
    X2 = Q.x
    Y2 = Q.y
    Z2 = Q.z
        
    Z1Z1 = Z1**2 %p
    Z2Z2 = Z2**2
    U1 = X1*Z2Z2
    U2 = X2*Z1Z1
    S1 = Y1*Z2*Z2Z2
    S2 = Y2*Z1*Z1Z1
    H = U2-U1
    I = (2*H)**2
    J = H*I
    r = 2*(S2-S1)
    V = U1*I
    X3 = r**2-J-2*V
    Y3 = r*(V-X3)-2*S1*J
    Z3 = ((Z1+Z2)**2-Z1Z1-Z2Z2)*H
    return Point_3d(X3,Y3,Z3)

In [589]:
def mult_jacobi(n,P,a,p) :
    result = P # This is our zero point
    for b in format(n,'b')[1:]: # Escape first bit which is 1 
        result = double_jacobi(result,a,p) # Doubling
        if b=='1':
            result = add_jacobi(result,P,a,p) # Addition
    return result

In [590]:
def mult_jacobi_old(n,P,a) :
    result = P # This is our zero point
    for b in format(n,'b')[1:]: # Escape first bit which is 1 
        result = double_jacobi_old(result,a) # Doubling
        if b=='1':
            result = add_jacobi_old(result,P,a) # Addition
    return result

In [598]:
def mult_jacobi_MD(n,P_MD,a,p,prec) :
    result = P_MD # This is our zero point
    for b in format(n,'b')[1:]: # Escape first bit which is 1 
        result = double_jacobi_MD(result,a,p,prec) # Doubling
        if b=='1':
            result = add_jacobi_MD(result,P_MD,a,p,prec) # Addition
    return result

## Testing Affine and Jacobi and Montgomery formulas :

In [653]:
a=11 # curve parameter
p=43 # Modulo
prec = 512 # Montgomery precision
P = Point(16,11) # Point(15,17) + Point(x=15, y=26) = Infinity
Q = Point(25,15)
n = 1763563563632213 # n = 140 -> P_3d.z = 0 -> How to convert it to Affine ?

P_3d = affine_to_jacobi(P,p)
Q_3d = affine_to_jacobi(Q,p)

P_MD = affine_to_jacobi_MD(P,p,prec)
Q_MD = affine_to_jacobi_MD(Q,p,prec)

In [654]:
# Test point doubling
p_aff = double_affine(P,a,p)
p_jac = double_jacobi(P_3d,a,p)
p_mon = double_jacobi_MD(P_MD,a,p,prec)

print("Affine     : ", p_aff)
print("Jacobi     : ",jacobi_to_affine(p_jac,p))
print("Montgomery : ",jacobi_MD_to_affine(p_mon,p,prec))

Affine     :  Point(x=25, y=28)
Jacobi     :  Point(x=25, y=28)
Montgomery :  Point(x=25, y=28)


In [655]:
# Test point addition
p_aff = add_affine(P,Q,a,p)
p_jac = add_jacobi(P_3d,Q_3d,a,p)
p_mon = add_jacobi_MD(P_MD,Q_MD,a,p,prec)

print("Affine     : ", p_aff)
print("Jacobi     : ",jacobi_to_affine(p_jac,p))
print("Montgomery : ",jacobi_MD_to_affine(p_mon,p,prec))

Affine     :  Point(x=16, y=32)
Jacobi     :  Point(x=16, y=32)
Montgomery :  Point(x=16, y=32)


In [656]:
# Test point multiplication :
p_aff = double_and_add(n,P,a,p)
p_jac = mult_jacobi(n,P_3d,a,p)
p_mon = mult_jacobi_MD(n,P_MD,a,p,prec)

print("Affine     : ", p_aff)
print("Jacobi     : ",jacobi_to_affine(p_jac,p))
print("Montgomery : ",jacobi_MD_to_affine(p_mon,p,prec))

Affine     :  Point(x=7, y=15)
Jacobi     :  Point(x=7, y=15)
Montgomery :  Point(x=7, y=15)


## Testing special cases (P+Q, 2P and nP equals to infinity)

In [657]:
P = Point(16,11)
Q = Point(x=16, y=32)

P_3d = affine_to_jacobi(P,p)
Q_3d = affine_to_jacobi(Q,p)

P_MD = affine_to_jacobi_MD(P,p,prec)
Q_MD = affine_to_jacobi_MD(Q,p,prec)

In [658]:
# Testing the case where P + Q = Infinity :

print("Affine     : ",add_affine(P,Q,a,p))
print("Jacobi     : ",jacobi_to_affine(add_jacobi(P_3d,Q_3d,a,p),p))
print("Montgomery : ",jacobi_MD_to_affine(add_jacobi_MD(P_MD,Q_MD,a,p,prec),p,prec))

Affine     :  None
Jacobi     :  None
Montgomery :  None


In [659]:
# Testing the case where Doubling P = Infinity :

P = Point(40,0)
P_3d = affine_to_jacobi(P,p)
P_MD = affine_to_jacobi_MD(P,p,prec)

print("Affine     : ",double_affine(P,a,p))
print("Jacobi     : ",jacobi_to_affine(double_jacobi(P_3d,a,p),p))
print("Montgomery : ",jacobi_MD_to_affine(double_jacobi_MD(P_MD,a,p,prec),p,prec))

Affine     :  None
Jacobi     :  None
Montgomery :  None


In [660]:
# Testing the case n*P = Infinity :
n = 35
P = Point(16,11)
P_3d = affine_to_jacobi(P,p)
P_MD = affine_to_jacobi_MD(P,p,prec)

print("Affine     : ",double_and_add(n,P,a,p))
print("Jacobi     : ",jacobi_to_affine(mult_jacobi(n,P_3d,a,p),p))
print("Montgomery : ",jacobi_MD_to_affine(mult_jacobi_MD(n,P_MD,a,p,prec),p,prec))

Affine     :  None
Jacobi     :  None
Montgomery :  None


In [670]:
from timeit import default_timer as timer
n = 287437123*34
print(n)
s = format(n,'b')
print(len(s))
print("1 : ",s.count('1'))
print("0 : ",s.count('0'))

x = '0b1101010101110101010000100000111101010101111110101011101'
print(int(x,2))

9772862182
34
1 :  13
0 :  21
30041548312477021


In [671]:
R1 = jacobi_to_affine(mult_jacobi(n,P_3d,a,p),p)
R2 = double_and_add(n,P,a,p)
print("Jacobi : ",R1)
print("Affine : ",R2)

Jacobi :  Point(x=31, y=40)
Affine :  Point(x=31, y=40)


In [663]:
start = timer()
R1 = mult_jacobi(n,P_3d,a,p)
result = jacobi_to_affine(R1,p)
end = timer()
print (end-start)
result

0.001151900039985776


Point(x=31, y=3)

In [664]:
start = timer()
R2 = double_and_add(n,P,a,p)
end = timer()
print (end-start)
print(R2)

0.0006682999664917588
None


## Verification using Big numbers (JcrypTool)

In [681]:
a = 6277101735386680763835789423207666416083908700390324961276
p = 6277101735386680763835789423207666416083908700390324961279
x = 4142864497108628997191135398145956424468169166144183778219
y = 4677937196124056539257824887551427911549894528556935618667
P = Point(x,y)
p_3d = affine_to_jacobi(P,p)
n = 1983249124
P_jCryptool = Point(271200204370615955000047308808338852259101967016874506119,1382724357982614913185871638334457544038335605528526572970
)

In [678]:
double_and_add(n,P,a,p) == P_jCryptool

True